# Load data

In [1]:
%%info

In [2]:
data_src = "s3a://udacity-dsnd/sparkify/sparkify_event_data.json"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1691568817964_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.json(data_src)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Process data for prediction

Filter out all not logged-in users

In [6]:
df = df.filter(df["userId"] != "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.createOrReplaceTempView("events")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
avg_session_len_per_user = spark.sql('''
    SELECT userId, AVG(length) AS avg_session_length FROM (
        SELECT userId, MAX(ts) - MIN(ts) AS length FROM events
        GROUP BY userId, sessionId
    )
    GROUP BY userId
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.functions import last, udf, pandas_udf, PandasUDFType
from pyspark.sql.types import FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
session_ends = df.sort("ts").groupby(df.userId, df.sessionId).agg(last(df.ts).alias("end"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
@pandas_udf(FloatType())
def time_bw_sessions(s: pd.Series) -> float:
    return s.diff().mean()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
PyArrow >= 1.0.0 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1691496157060_0001/container_1691496157060_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py", line 337, in pandas_udf
    require_minimum_pyarrow_version()
  File "/mnt/yarn/usercache/livy/appcache/application_1691496157060_0001/container_1691496157060_0001_01_000001/pyspark.zip/pyspark/sql/pandas/utils.py", line 63, in require_minimum_pyarrow_version
    ) from raised_error
ImportError: PyArrow >= 1.0.0 must be installed; however, it was not found.



In [18]:
avg_time_bw_sessions = session_ends.sort("sessionId").groupby("userId").agg(time_bw_sessions("end"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'time_bw_sessions' is not defined
Traceback (most recent call last):
NameError: name 'time_bw_sessions' is not defined



In [11]:
time_per_day = spark.sql('''
    SELECT events.userId, ROUND(COUNT(*) / first(a.days), 2) AS pages_per_day
    FROM events
    JOIN (
        SELECT userId, GREATEST(1, ROUND((MAX(ts) - MIN(ts)) / 3600 / 24 / 1000, 2)) AS days FROM events
        GROUP BY userId
    ) AS a ON events.userId = a.userId
    GROUP BY events.userId
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']

Find how many times a user visits each page

In [12]:
pages_per_user_by_type = spark.sql('''
    SELECT userId, page, COUNT(*) AS page_visits
    FROM events
    GROUP BY userId, page
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.sql.functions import coalesce, sum as fsum, lit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
pages_per_user_by_type = pages_per_user_by_type.groupby("userId").pivot("page").sum("page_visits").na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
demographic = spark.sql('''
    SELECT userId, any_value(gender) AS gender
    FROM events
    GROUP BY userId
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol="gender", outputCol="genderIndexed", handleInvalid="skip")
model = stringIndexer.fit(demographic)
demographic_indexed = model.transform(demographic)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
final = pages_per_user_by_type.join(time_per_day, on="userId").join(avg_session_len_per_user, on="userId").join(demographic_indexed, on="userId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
from pyspark.sql.functions import isnan, when, count, col

final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+----------+---------------+------+-------------------------+---------+-----+----+----+-----+------+--------+--------+-----------+-------------+--------+----------------+-------------------+--------------+-----------+---------+-------+-------------+------------------+------+
|userId|About|Add Friend|Add to Playlist|Cancel|Cancellation Confirmation|Downgrade|Error|Help|Home|Login|Logout|NextSong|Register|Roll Advert|Save Settings|Settings|Submit Downgrade|Submit Registration|Submit Upgrade|Thumbs Down|Thumbs Up|Upgrade|pages_per_day|avg_session_length|gender|
+------+-----+----------+---------------+------+-------------------------+---------+-----+----+----+-----+------+--------+--------+-----------+-------------+--------+----------------+-------------------+--------------+-----------+---------+-------+-------------+------------------+------+
|     0|    0|         0|              0|     0|                        0|        0|    0|   0|   0|    0|     0|       0|       0|  

# Modeling

In [18]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Including visits to the "Cancel" page would likely introduce data leakage. And we want to predict churn before the user cancels so let's drop those columns

In [19]:
feature_cols = list(set(final.columns) - set(["userId", "Cancel", "Cancellation Confirmation", "gender"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
feature_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Home', 'avg_session_length', 'Logout', 'Downgrade', 'Thumbs Down', 'Login', 'Help', 'Save Settings', 'Upgrade', 'Roll Advert', 'Submit Upgrade', 'Submit Registration', 'About', 'Error', 'Settings', 'Submit Downgrade', 'Register', 'Add Friend', 'Thumbs Up', 'Add to Playlist', 'genderIndexed', 'pages_per_day', 'NextSong']

In [21]:
vec_assembler = VectorAssembler(outputCol="features")
vec_assembler.setInputCols(feature_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VectorAssembler_3593eed58e19

In [22]:
from pyspark.ml.classification import GBTClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
data = vec_assembler.transform(final).select("features", final["Cancellation Confirmation"].alias("label"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
gbt = GBTClassifier(maxIter=5, maxDepth=5, labelCol="label", seed=42,
    leafCol="leafId")

model = gbt.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
# Evaluate model on test instances and compute test error
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
from pyspark.sql.functions import when

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
confusion_mat = predictions.select(
    fsum(when(predictions.label + predictions.prediction == 0, 1).otherwise(0)).alias("TN"),
    fsum(when(predictions.label + predictions.prediction == 2, 1).otherwise(0)).alias("TP"),
    fsum(when(predictions.label > predictions.prediction, 1).otherwise(0)).alias("FN"),
    fsum(when(predictions.label < predictions.prediction, 1).otherwise(0)).alias("FP")
).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
precision = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FP"])
recall = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FN"])
precision, recall

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.7926267281105991, 0.5702917771883289)

Not the best results, let's try some cross validation and grid-searching.

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
gbt = GBTClassifier(maxIter=5, maxDepth=5, labelCol="label", leafCol="leafId")
grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10, 50]) \
    .addGrid(gbt.maxDepth, [2, 5, 10]) \
    .build()
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator)

cvModel = cv.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-92:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3511



In [93]:
cvModel.bestModel.getMaxDepth()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5

In [94]:
cvModel.bestModel.getMaxIter()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50

Much better scores after CV.

In [95]:
# Evaluate model on test instances and compute test error
predictions = cvModel.bestModel.transform(testData)
confusion_mat = predictions.select(
    fsum(when(predictions.label + predictions.prediction == 0, 1).otherwise(0)).alias("TN"),
    fsum(when(predictions.label + predictions.prediction == 2, 1).otherwise(0)).alias("TP"),
    fsum(when(predictions.label > predictions.prediction, 1).otherwise(0)).alias("FN"),
    fsum(when(predictions.label < predictions.prediction, 1).otherwise(0)).alias("FP")
).collect()
precision = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FP"])
recall = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FN"])
precision, recall

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.8353140916808149, 0.6525198938992043)

In [30]:
gbt = GBTClassifier(maxIter=5, maxDepth=5, labelCol="label", leafCol="leafId")
grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [50, 75, 100]) \
    .addGrid(gbt.maxDepth, [4, 5, 6]) \
    .build()
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator, parallelism=9)

cvModel = cv.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [31]:
cvModel.bestModel.getMaxDepth()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [32]:
cvModel.bestModel.getMaxIter()

VBox()

Exception in thread cell_monitor-30:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

In [34]:
from pyspark.sql.functions import when

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Evaluate model on test instances and compute test error
predictions = cvModel.bestModel.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-42:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [ ]:
confusion_mat = predictions.select(
    fsum(when(predictions.label + predictions.prediction == 0, 1).otherwise(0)).alias("TN"),
    fsum(when(predictions.label + predictions.prediction == 2, 1).otherwise(0)).alias("TP"),
    fsum(when(predictions.label > predictions.prediction, 1).otherwise(0)).alias("FN"),
    fsum(when(predictions.label < predictions.prediction, 1).otherwise(0)).alias("FP")
).collect()
precision = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FP"])
recall = confusion_mat[0]["TP"] / (confusion_mat[0]["TP"] + confusion_mat[0]["FN"])
precision, recall

Exception in thread cell_monitor-33:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-34:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



(0.8694835680751174, 0.6144658261446583)

Better precision, but slightly worse recall. The model is likely better overall, but let's tune the threshold to try to get better recall. Let's convert the predictions into a pandas dataframe for easy calculation.

In [50]:
predictions_df = predictions.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-49:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [55]:
predictions.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['features', 'label', 'rawPrediction', 'probability', 'prediction', 'leafId']

Exception in thread cell_monitor-54:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [88]:
import numpy as np
thresholds = np.linspace(0.15, 0.3, 100)
raw_probs = np.stack(predictions_df["probability"].map(lambda x: x.toArray()).values)[:, 1]
results = {"precision": [], "recall": [], "threshold": []}
for i in list(thresholds):
    preds = raw_probs > i
    tn = (preds + predictions_df["label"] == 0).sum()
    tp = (preds + predictions_df["label"] == 2).sum()
    fn = (preds < predictions_df["label"]).sum()
    fp = (preds > predictions_df["label"]).sum()
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    results["threshold"].append(i)
    results["precision"].append(precision)
    results["recall"].append(recall)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-87:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [94]:
idx = len(np.array(results["recall"])[np.array(results["recall"]) > 0.8])-1
results["threshold"][idx]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.22727272727272727

Exception in thread cell_monitor-93:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



We get a prediction threshold of 0.22727272727272727 instead of the default 0.5.

Using this new threshold, we now hav a model with a recall of 80% on the test set, meaning that at least 80% of all true positives will be recovered. This should help us detect customers who may churn. There will be an increase in false positives as well so whatever marketing strategy is employed later will need to take that into account.